In [79]:
import json
import sys
from rich import print as rprint

from pathlib import Path
from itertools import islice

nb_dir = Path.cwd()

project_root = nb_dir.parent.parent

sys.path.insert(0, str(project_root))


In [ ]:
people_records_prepped_file = Path(project_root / "data/people/people_records_prepped.json")
books2people_prepped_file = Path(project_root / "data/people/books2people_prepped.json")
folder_matched = Path(project_root / "data/matched")

validation_log_file = Path(project_root / "data/logs/validation_failed_log.json")

In [81]:
people_records = []
books2people_records = []

with open(people_records_prepped_file, "r") as f:
    people_records = json.load(f)

with open(books2people_prepped_file, "r") as f:
    books2people_records = json.load(f)

books2people_dict = {entry["composite_id"]: entry for entry in books2people_records}
people_dict = {person["unified_id"]: person for person in people_records}


In [ ]:
for file in folder_matched.iterdir():
    if not file.exists():
        raise FileNotFoundError(f"{file} doesn't exist!")

    with open(file, "r") as f:
        books = json.load(f)
        #rprint(books)

        for composite_id, book in books.items():
            authors = book["parsed_entry"]["authors"]
            editors = book["parsed_entry"]["editors"]
            contributors = book["parsed_entry"]["contributors"]
            translator = book["parsed_entry"]["translator"]
            is_translation = book["parsed_entry"]["is_translation"]
            authors_exp = len(authors)
            editors_exp = len(editors)
            contributors_exp = len(contributors)
            translator_exp = 1 if translator else 0


0

0

0

0

0

...

0

0

0

0

0